
# Data Preparation

In [2]:
import pandas as pd
import numpy as np

In [3]:
# Load the dataset
df = pd.read_csv('dataset/Crops_District_Production.csv')
# basic Information
print("Dataset shape:", df.shape)
print("\nFirst few rows:")
print(df.head())
print("\nColumn info:")
print(df.info())
# Check for any missing values
print("\nMissing values:")
print(df.isnull().sum())

Dataset shape: (11002, 6)

First few rows:
         date  state    district   crop_type  crop_species  production
0  2017-01-01  Johor  Batu Pahat  cash_crops       cassava       920.5
1  2017-01-01  Johor  Batu Pahat  cash_crops    groundnuts         0.0
2  2017-01-01  Johor  Batu Pahat  cash_crops    sweet_corn         0.0
3  2017-01-01  Johor  Batu Pahat  cash_crops  sweet_potato       350.0
4  2017-01-01  Johor  Batu Pahat  cash_crops           yam       395.4

Column info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11002 entries, 0 to 11001
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   date          11002 non-null  object 
 1   state         11002 non-null  object 
 2   district      11002 non-null  object 
 3   crop_type     11002 non-null  object 
 4   crop_species  11002 non-null  object 
 5   production    11002 non-null  float64
dtypes: float64(1), object(5)
memory usage: 515.8+ KB
None

Missi

In [6]:
# Group by state, district, crop_type and find top 5 species by production
def transform_to_top5_format(df):
    """
    Transform crop data to show top 5 species per district/crop_type
    Modern pandas approach that avoids all warnings
    """
    # Sort by production in descending order
    df_sorted = df.sort_values(['state', 'district', 'crop_type', 'production'], 
                              ascending=[True, True, True, False])
    
    # Get top 5 for each group and explicitly create a copy
    top5_df = df_sorted.groupby(['state', 'district', 'crop_type']).head(5).copy()
    
    # Create ranking within each group using .loc to avoid warning
    top5_df.loc[:, 'rank'] = top5_df.groupby(['state', 'district', 'crop_type']).cumcount() + 1
    
    # Pivot to get the desired format
    result = top5_df.pivot_table(
        index=['state', 'district', 'crop_type'],
        columns='rank',
        values='crop_species',
        aggfunc='first'
    ).reset_index()
    
    # Rename columns to crop_rank_1, crop_rank_2, etc.
    result.columns.name = None  # Remove the 'rank' column name
    new_columns = ['state', 'district', 'crop_type'] + [f'crop_rank_{i}' for i in range(1, 6)]
    result.columns = new_columns[:len(result.columns)]
    
    return result

# Transform the data
df = transform_to_top5_format(df)

# Display results
print("Transformed dataset shape:", df.shape)
print("\nFirst 10 rows:")
print(df.head(10))

Transformed dataset shape: (903, 8)

First 10 rows:
   state     district         crop_type           crop_rank_1  \
0  Johor   Batu Pahat        cash_crops               cassava   
1  Johor   Batu Pahat             fruit             pineapple   
2  Johor   Batu Pahat             herbs  fragrant_lemon_grass   
3  Johor   Batu Pahat  industrial_crops               coconut   
4  Johor   Batu Pahat            spices       calamondin_lime   
5  Johor   Batu Pahat         vegetable             long_bean   
6  Johor  Johor Bahru        cash_crops               cassava   
7  Johor  Johor Bahru             fruit                banana   
8  Johor  Johor Bahru             herbs  fragrant_lemon_grass   
9  Johor  Johor Bahru  industrial_crops               coconut   

         crop_rank_2       crop_rank_3       crop_rank_4   crop_rank_5  
0                yam      sweet_potato        groundnuts    sweet_corn  
1             papaya            durian            banana         guava  
2            

In [7]:
# Combine all csv data into single dataset:
print("Load all CSV files...")

# disctrict data and corps
print(f"District data and Corps shape: {df.shape}")
# Climate data (rainfall and temperature)
df_climate = pd.read_csv('dataset/district rainfall temperature.csv')
print(f"Climate data shape: {df_climate.shape}")

# Elevation and slope data
df_elevation = pd.read_csv('dataset/elevation and slope dataset 155 districts.csv')
print(f"Elevation data shape: {df_elevation.shape}")

Load all CSV files...
District data and Corps shape: (903, 8)
Climate data shape: (155, 4)
Elevation data shape: (155, 9)


in here, we see that the disctrict data is bigger than the climate and elevation data because of multiple crop_type on each district.

In [ ]:
# Check column names and normalize it
print("\n1. Climate data:")
# rename the columns to avg_rainfall_mm and avg_temp_c
df_climate.columns = ['state', 'district', 'avg_rainfall_mm', 'avg_temp_c']
print(df_climate.columns.tolist())
print(f"Unique state-district combinations: {df_climate.shape[0]}")

print("\n2. Elevation data:")
print(df_elevation.columns.tolist())
print(f"Unique state-district combinations: {df_elevation.shape[0]}")


1. Climate data:
['state', 'district', 'avg_rainfall_mm', 'avg_temp_c']
Unique state-district combinations: 155

2. Elevation data:
['state', 'district', 'elev_mean', 'elev_min', 'elev_max', 'slope_mean', 'slope_min', 'slope_max', 'slope_std']
Unique state-district combinations: 155


In [31]:
# data cleaning
# Clean column names (remove extra spaces)
df_climate.columns = df_climate.columns.str.strip()
df_elevation.columns = df_elevation.columns.str.strip()

# Standardize state and district names (remove extra spaces, standardize case)
for df in [df_crops, df_climate, df_elevation]:
    df['state'] = df['state'].str.strip().str.title()
    df['district'] = df['district'].str.strip().str.title()

print("Data cleaned - standardized state/district names")

Data cleaned - standardized state/district names


In [ ]:
# Merging dataset
print("MERGING DATASETS")

# Step 1: Start with main crop production data (using existing df)
combined_df = df.copy()
print(f"Starting with crop data: {combined_df.shape}")

# Step 2: Merge with climate data
combined_df = pd.merge(
    combined_df, 
    df_climate, 
    on=['state', 'district'], 
    how='left'
)
print(f"After adding climate data: {combined_df.shape}")

# Step 3: Merge with elevation data
combined_df = pd.merge(
    combined_df, 
    df_elevation, 
    on=['state', 'district'], 
    how='left'
)
print(f"After adding elevation data: {combined_df.shape}")


MERGING DATASETS
Starting with crop data: (903, 8)
After adding climate data: (903, 10)
After adding elevation data: (903, 17)


check the combined data

In [17]:
print("head")
combined_df.head()

head


,state,district,crop_type,crop_rank_1,crop_rank_2,crop_rank_3,crop_rank_4,crop_rank_5,average rainfall (mm),average temp (C),elev_mean,elev_min,elev_max,slope_mean,slope_min,slope_max,slope_std
0,Johor,Batu Pahat,cash_crops,cassava,yam,sweet_potato,groundnuts,sweet_corn,7.589,27.337,21.366024,-8.0,567.0,2.068859,0.0,46.385242,3.134855
1,Johor,Batu Pahat,fruit,pineapple,papaya,durian,banana,guava,7.589,27.337,21.366024,-8.0,567.0,2.068859,0.0,46.385242,3.134855
2,Johor,Batu Pahat,herbs,fragrant_lemon_grass,cekur,lemon_myrtle,aloe_vera,basil,7.589,27.337,21.366024,-8.0,567.0,2.068859,0.0,46.385242,3.134855
3,Johor,Batu Pahat,industrial_crops,coconut,coffee,roselle,mushroom,areca_nut,7.589,27.337,21.366024,-8.0,567.0,2.068859,0.0,46.385242,3.134855
4,Johor,Batu Pahat,spices,calamondin_lime,lemon_grass,greater_galangal,pink_cone_ginger,lime,7.589,27.337,21.366024,-8.0,567.0,2.068859,0.0,46.385242,3.134855


In [21]:
print("random")
combined_df.iloc[6:10]

random


,state,district,crop_type,crop_rank_1,crop_rank_2,crop_rank_3,crop_rank_4,crop_rank_5,average rainfall (mm),average temp (C),elev_mean,elev_min,elev_max,slope_mean,slope_min,slope_max,slope_std
6,Johor,Johor Bahru,cash_crops,cassava,yellow_sugar_cane,sweet_potato,yam,sweet_corn,NaN,NaN,31.036336,-18.0,176.0,2.484138,0.0,22.806465,1.74778
7,Johor,Johor Bahru,fruit,banana,pineapple,durian,rambutan,papaya,NaN,NaN,31.036336,-18.0,176.0,2.484138,0.0,22.806465,1.74778
8,Johor,Johor Bahru,herbs,fragrant_lemon_grass,aloe_vera,basil,belalai_gajah,betel_vine,NaN,NaN,31.036336,-18.0,176.0,2.484138,0.0,22.806465,1.74778
9,Johor,Johor Bahru,industrial_crops,coconut,mushroom,areca_nut,coffee,honey_bee,NaN,NaN,31.036336,-18.0,176.0,2.484138,0.0,22.806465,1.74778


In [18]:
print("tail")
combined_df.tail()

tail


,state,district,crop_type,crop_rank_1,crop_rank_2,crop_rank_3,crop_rank_4,crop_rank_5,average rainfall (mm),average temp (C),elev_mean,elev_min,elev_max,slope_mean,slope_min,slope_max,slope_std
898,W.P. Labuan,W.P. Labuan,cash_crops,cassava,yellow_sugar_cane,sweet_corn,NaN,NaN,11.079,27.325,NaN,NaN,NaN,NaN,NaN,NaN,NaN
899,W.P. Labuan,W.P. Labuan,fruit,banana,jackfruit,mango,rambutan,cempedak,11.079,27.325,NaN,NaN,NaN,NaN,NaN,NaN,NaN
900,W.P. Labuan,W.P. Labuan,industrial_crops,coconut,NaN,NaN,NaN,NaN,11.079,27.325,NaN,NaN,NaN,NaN,NaN,NaN,NaN
901,W.P. Labuan,W.P. Labuan,spices,calamondin_lime,lemon_grass,NaN,NaN,NaN,11.079,27.325,NaN,NaN,NaN,NaN,NaN,NaN,NaN
902,W.P. Labuan,W.P. Labuan,vegetable,mustard,water_spinach,chinese_spinach,sweet_shoot,cucumber,11.079,27.325,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
print("FINAL COMBINED DATASET")
print(f"Final dataset shape: {combined_df.shape}")
print(f"Columns: {combined_df.columns}")

print("\nMissing values in combined dataset:")
missing_values = combined_df.isnull().sum()
missing_values = missing_values[missing_values > 0]
if len(missing_values) > 0:
    print(missing_values)
else:
    print("No missing values!")

FINAL COMBINED DATASET
Final dataset shape: (903, 17)
Columns: Index(['state', 'district', 'crop_type', 'crop_rank_1', 'crop_rank_2',
       'crop_rank_3', 'crop_rank_4', 'crop_rank_5', 'average rainfall (mm)',
       'average temp (C)', 'elev_mean', 'elev_min', 'elev_max', 'slope_mean',
       'slope_min', 'slope_max', 'slope_std'],
      dtype='object')

Missing values in combined dataset:
crop_rank_2               41
crop_rank_3               50
crop_rank_4              111
crop_rank_5              189
average rainfall (mm)    730
average temp (C)         730
elev_mean                130
elev_min                 130
elev_max                 130
slope_mean               130
slope_min                130
slope_max                130
slope_std                130
dtype: int64


In [45]:
# Since there are many missing values in avg_rainfall_mm and avg_temp_c, we will use the median of their state to fill in the missing values.
combined_df['avg_rainfall_mm'] = combined_df.groupby('state')['avg_rainfall_mm'].transform(lambda x: x.fillna(x.median()))
combined_df['avg_temp_c'] = combined_df.groupby('state')['avg_temp_c'].transform(lambda x: x.fillna(x.median()))


print("\nMissing values in combined dataset:")
missing_values = combined_df.isnull().sum()
missing_values = missing_values[missing_values > 0]
if len(missing_values) > 0:
    print(missing_values)
else:
    print("No missing values!")


# Check which specific state-district combinations are missing climate data
missing_climate = combined_df[combined_df['avg_rainfall_mm'].isnull()]
print("State-District combinations missing climate data:")
print(missing_climate[['state', 'district']].drop_duplicates())
print(f"Number of unique combinations missing: {missing_climate[['state', 'district']].drop_duplicates().shape[0]}")


Missing values in combined dataset:
avg_rainfall_mm    26
avg_temp_c         26
dtype: int64
State-District combinations missing climate data:
            state                district
32         Labuan             W.P. Labuan
54          Perak           Batang Padang
55          Perak           Muallim (New)
56          Perak             Hilir Perak
57          Perak       Bagan Datuk (New)
58          Perak              Hulu Perak
59          Perak                  Kampar
60          Perak                  Kerian
61          Perak                   Kinta
62          Perak           Kuala Kangsar
63          Perak        Larut Dan Matang
64          Perak                 Manjung
65          Perak            Perak Tengah
67   Pulau Pinang              Barat Daya
68   Pulau Pinang  Seberang Perai Selatan
69   Pulau Pinang   Seberang Perai Tengah
70   Pulau Pinang    Seberang Perai Utara
71   Pulau Pinang              Timur Laut
147     Trengganu                   Besut
148     Trenggan